# EV_charginging_facts

***Table of Contents***
1. Ev Car Poulation 
    - Information about the dataset
    - Extracting 
        1. Saving to a df
        2. Extracting the columns that we need and saving as a new df 
        3. Removing null values that we do not need 
    - Transforming 
        1. Adding registiration date column
        2. Splitting the vehichle location to lat and long 
        3. Adding country column:
        4. Adding current registiraction status
        5. Adding quantity field
        6. Adding pk field 
        7. Adding SCD columns
        8. Transforming column names 
2. EV Stations Data-Set
    - Information about the dataset
    - Extracting 
        1. Saving to a df
        2. Extracting the columns that we need and saving as a new df 
        3. Removing null values and filtering out unnecessary date that we do not need 
    - Transforming 
        1. Transforming status code to understandable format for users <br>
        2. Adding the quantity column<br>
        3. Extracting dollor amount from the price column and transforming to float<br>
        4. Adding currency column<br>
        5. Adding PK column<br>
        6. Adding SK Column <br>
        7. Transforming column names <br>
        8. Transforming activity_date column to date type <br>
        9. Transforming column names <br>
3.  Combining two data set to create the ev_charging_fact table
4.  Creating the Location data frame 
    a. extracting location info and save as a df<br>
    b. populate region info from a look up table<br>
    c. create a pk column<br>
    e. adding scd columns<br>
    f. merge to the result df to bring pk column to the fact df<br>
5.  Creating the date data frame 
    a. extracting location info and save as a df<br>
    b. spearate date, month ,year<br>
    c. create a pk column<br>
6.  Creating the fact table df 
7.  Saving cumulative df 

***Libraries***

In [164]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import  pandas as pd 

## 1  EV Car Population Data 

Information about the data set

* last updated at March 18, 2023
The VIN is assigned to a specific vehicle model by the manufacturer, and it remains with the vehicle throughout its lifetime. In contrast, the DOL Vehicle ID is assigned by the Washington State Department of Licensing (DOL) to a specific vehicle when it is registered and titled in Washington State. So the VIN identifies the make, model, and features of a particular vehicle, while the DOL Vehicle ID identifies the vehicle as registered in the state of Washington

we need to think this table as scd 3 because the records that this table can hardly change, for example VIN or vehicle id these identifications are important and they almost never change but only field that can change is the registiration status. In our data we do not have field for it but we can create one and assume that we have this field to keep things simple. Logic would be when we refreshing we will check
* do we have the record of the specific car in our new data
    * if we do we skip it 
    * else we set the current reg status not_active and prev status to active 


* total populated and extracted data fields :19

### 1.1 ---------------------extracting the data -----------

1. Loading the data set in a data frame

In [165]:
ev_pop = pd.read_csv('data/Electric_Vehicle_Population_Data.csv')
ev_pop.head(2)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJ3E1EB2J,Suffolk,Suffolk,VA,23435.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,NaN,476647986,POINT (-76.42443 36.8752),NaN,5.180008e+10
1,5YJ3E1ECXL,Yakima,Yakima,WA,98908.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,14.0,103490145,POINT (-120.56916 46.58514),PACIFICORP,5.307700e+10


2 Exploring: we see that we have 121978 rows  and 17 columns

In [166]:
ev_pop.shape

(121978, 17)

In [167]:
ev_pop.columns

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract'],
      dtype='object')

3. extracting the information that we need and saving to a different data frame 

In [168]:
df_ev_pop = ev_pop[['VIN (1-10)', 'DOL Vehicle ID', 'Make','Model','Vehicle Location', 'City', 'State', 'Model Year']]

4. Removing Null Values 

In [169]:
df_ev_pop = df_ev_pop.loc[df_ev_pop['City'].notnull()]
df_ev_pop = df_ev_pop.loc[df_ev_pop['City'].notna()]

### 1.2 ------------------- Transforming the Data ---------

**1. adding registiration date column:** (this field does not exsist but we need it because we are missing a date dimension) and we are basing registiration date from model year. we are naming this field as activity date because we are going to use this field for our merge <br>


In [170]:
df_ev_pop["activity_date"] = "01/01/" + df_ev_pop['Model Year'].astype(str)

now we need to change the data type ad date type

In [171]:
df_ev_pop["activity_date"]= df_ev_pop["activity_date"].astype('datetime64[D]')

let's drop make 

we can now drop model year since we have it as a date 

In [172]:
df_ev_pop = df_ev_pop.drop(columns=['Model Year'])

**2. Splitting the vehichle location to lat and long**: We need these fields to spearated so we can create viz or run some analytics queries and we need to make sure 
that they are stored as float incase we need to calculate the distance in between two location

In [173]:
df_ev_pop[["val", "Latitude", "Longitude"]] = df_ev_pop['Vehicle Location'].str.split(' ', expand=True)
# removing paranthesis 
df_ev_pop["Latitude"] =  df_ev_pop["Latitude"].str[1:]
df_ev_pop["Longitude"] =df_ev_pop["Longitude"].str.replace(')',"")
df_ev_pop["Longitude"]= df_ev_pop["Longitude"].str.strip()
df_ev_pop["Latitude"] =  df_ev_pop["Latitude"].str.strip()
df_ev_pop.head()

,VIN (1-10),DOL Vehicle ID,Make,Model,Vehicle Location,City,State,activity_date,val,Latitude,Longitude
0,5YJ3E1EB2J,476647986,TESLA,MODEL 3,POINT (-76.42443 36.8752),Suffolk,VA,2018-01-01,POINT,-76.42443,36.8752
1,5YJ3E1ECXL,103490145,TESLA,MODEL 3,POINT (-120.56916 46.58514),Yakima,WA,2020-01-01,POINT,-120.56916,46.58514
2,WA1LAAGE7M,144941534,AUDI,E-TRON,POINT (-120.56916 46.58514),Yakima,WA,2021-01-01,POINT,-120.56916,46.58514
3,5YJ3E1EA1K,168513922,TESLA,MODEL 3,POINT (-79.4172 36.58598),Danville,VA,2019-01-01,POINT,-79.4172,36.58598
4,1FADP5CU9E,150749378,FORD,C-MAX,POINT (-76.21549 36.92478),Norfolk,VA,2014-01-01,POINT,-76.21549,36.92478


In [174]:
df_ev_pop["Longitude"]= df_ev_pop["Longitude"].astype(float)
df_ev_pop["Latitude"] =  df_ev_pop["Latitude"].astype(float)

In [175]:
df_ev_pop = df_ev_pop.drop(columns=['val'])
df_ev_pop = df_ev_pop.drop(columns=['Vehicle Location'])

**3. Adding country column:** We need this field for the merge and it is missing in the original data. We know that all this locations are in united states. so we populate all the rows with US

In [176]:
df_ev_pop["country"]= 'US'

**4. Adding current registiraction status :** this field does not exsist but we know that people move and when they move the registiration to that state might move to another state and you need to move your registiration. Therefore we are creating this field to keep track of this change

In [177]:
df_ev_pop["current_reg_status"]= 'active'

**5. Adding quantity field:** we want to keep track how many cars in that region so we are adding the quantity field

In [178]:
df_ev_pop["quantity"]= int(1)

**6. Adding pk field :** I checked to see if the the records are unique and all of theme were. I used DOL Vehicle ID to check. Since we know we have unique cars we can create new pk number for the cars. 

In [179]:
df_ev_pop =df_ev_pop.reset_index(drop=True)
df_ev_pop['ev_car_pop_id'] =  df_ev_pop.index+1

**7. Adding SCD columns:** Since we know that we do not need to keep track of anything other than the registiration status we can add scd type 3 columns.We are adding prev_reg_status column to keep track of the activty status and we will add the updated_at column to see when we update the registiration status.

In [180]:
df_ev_pop["prev_reg_status"] = ''

We can use current date to show the date that acitivity date has been adjusted. We will have registiration date and we will have updated at so this will give you the enough historical information about the this table

In [181]:
df_ev_pop["reg_sta_updated_at"]=  pd.Series(dtype='datetime64[ns]')

8. Transforming column names : some of the columns have empty spots and we also want to show that this dv-pop data so we add 

In [182]:
df_ev_pop.columns

Index(['VIN (1-10)', 'DOL Vehicle ID', 'Make', 'Model', 'City', 'State',
       'activity_date', 'Latitude', 'Longitude', 'country',
       'current_reg_status', 'quantity', 'ev_car_pop_id', 'prev_reg_status',
       'reg_sta_updated_at'],
      dtype='object')

In [183]:
df_ev_pop.rename(columns={'DOL Vehicle ID': 'dol_vehicle_id', 
                          'VIN (1-10)': 'vehicle_vin', 
                          'Make': 'vehicle_make',
                          'Latitude': 'latitude',
                          'Longitude': 'longitude',
                          'City': 'city',
                          'State': 'state',
                          'Model': 'vehicle_model',
                          'quantity': 'vehicle_pop_quantity',
                          }, inplace=True)

In [184]:
df_ev_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121976 entries, 0 to 121975
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   vehicle_vin           121976 non-null  object        
 1   dol_vehicle_id        121976 non-null  int64         
 2   vehicle_make          121976 non-null  object        
 3   vehicle_model         121872 non-null  object        
 4   city                  121976 non-null  object        
 5   state                 121976 non-null  object        
 6   activity_date         121976 non-null  datetime64[ns]
 7   latitude              121946 non-null  float64       
 8   longitude             121946 non-null  float64       
 9   country               121976 non-null  object        
 10  current_reg_status    121976 non-null  object        
 11  vehicle_pop_quantity  121976 non-null  int64         
 12  ev_car_pop_id         121976 non-null  int64         
 13 

our final data siz

In [185]:
df_ev_pop.shape

(121976, 15)

-----------------------

# 2 ev_station data set 

* "Access Code": This tells us if the station is public or private
* "Status Code": 

### 2. extracting the data -------------------------

**1. Saving to a df:**

In [186]:
ev_station = pd.read_csv('data/alt_fuel_stations.csv')
ev_station.head(5)

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,MD,NaN,NaN,NaN
1,CNG,PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Ave, left on...",Atlanta,GA,30303,NaN,770-350-3000,E,...,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN,NaN
2,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324,NaN,NaN,E,...,NaN,NaN,NaN,NaN,Q,3000,LD,NaN,NaN,NaN
3,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,HD,NaN,NaN,NaN
4,CNG,Clean Energy - Texas Department of Transportation,7721A Washington St,"I-10, Washington Ave exit, 1.5 blocks to the s...",Houston,TX,77007,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,Q,3000 3600,MD,NaN,NaN,NaN


In [187]:
ev_station.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

**2. Extracting the columns that we need and saving as a new df:**

In [188]:
ev_station_v1 = ev_station[['ID','Status Code', "Station Name", 'Fuel Type Code', "Access Code",  "City", "State","Country", "Latitude", "Longitude", "Open Date", 'EV Pricing'] ]


In [189]:
len(ev_station_v1)

56800

**3. Removing null values and not nessary rows to limit unnessary data:**

In [190]:
ev_station_v1 = ev_station_v1.loc[ev_station_v1['City'].notnull()]
ev_station_v1 = ev_station_v1.loc[ev_station_v1['City'].notna()]

In [191]:
ev_station_v1.head()

,ID,Status Code,Station Name,Fuel Type Code,Access Code,City,State,Country,Latitude,Longitude,Open Date,EV Pricing
0,17,E,Spire - Montgomery Operations Center,CNG,private,Montgomery,AL,US,32.367916,-86.267021,2010-12-01,NaN
1,42,E,PS Energy - Atlanta,CNG,public,Atlanta,GA,US,33.745843,-84.398837,1994-07-15,NaN
2,45,E,Metropolitan Atlanta Rapid Transit Authority,CNG,private,Atlanta,GA,US,33.821911,-84.367461,1996-12-15,NaN
3,64,E,United Parcel Service,CNG,private,Atlanta,GA,US,33.760256,-84.543822,1997-01-01,NaN
4,72,E,Clean Energy - Texas Department of Transportation,CNG,public,Houston,TX,US,29.779918,-95.435385,1996-12-15,NaN


fuel type cng = Compressed Natural Gas (CNG) is a gasoline and diesel fuel alternative consisting primarily of methane. The gas is associated with other fossil fuels (coal or oil) and is created by methanogenic organisms in landfill

E85= E85 (or flex fuel) is a term that refers to high-level ethanol-gasoline blends containing 51% to 83% ethanol, depending on geography and season (see Fuel Properties and E85 Flex Fuel Specification).

ELEC : electric 

LPG= LNG is natural gas in its liquid form. 

BD: ?

HY : Hydrogen Fueling

In [192]:
ev_station_v1["Fuel Type Code"].unique()

array(['CNG', 'E85', 'ELEC', 'LNG', 'LPG', 'BD', 'HY'], dtype=object)

since we are interested in ev chars we do not need other charging stations information, we filter them out and then drop the fuel type code 

In [193]:
ev_station_v1 = ev_station_v1.loc[ev_station_v1["Fuel Type Code"]=='ELEC']
# ev_station_v3 = ev_station_v2.loc[ev_station_v1["State"]=='WA']

In [194]:
ev_station_v1 = ev_station_v1.drop(columns=["Fuel Type Code"])

In [195]:
ev_station_v1.columns

Index(['ID', 'Status Code', 'Station Name', 'Access Code', 'City', 'State',
       'Country', 'Latitude', 'Longitude', 'Open Date', 'EV Pricing'],
      dtype='object')

### 2.3  transforming --------------------------------------

**1. Tranforming status code to user friendly print**

In [196]:
ev_station_v1['Status Code']= ['not_yet_accessible' if x == 'P' else 'temprorally_closed'  if x =='T'else 'open' for x in ev_station_v1['Status Code']]

**2. Adding the quantity column**

In [197]:
ev_station_v1["ev_cs_quantity"]= int(1)

**3. Extracting dollor amount from the price column and transforming to float**

In [198]:
ev_station_v1['EV Pricing'].head()

159     NaN
160    Free
161     NaN
162     NaN
163     NaN
Name: EV Pricing, dtype: object

In [199]:
#define function for conditional formatting
def cond_formatting(x):
    if x ==  'Free':
        return '$0'
    else:
        return x

In [200]:
ev_station_v1['EV Pricing']=ev_station_v1['EV Pricing'].apply(cond_formatting)

In [201]:
ev_station_v1['EV Pricing'].head()

159    NaN
160     $0
161    NaN
162    NaN
163    NaN
Name: EV Pricing, dtype: object

In [202]:
df_price = ev_station_v1['EV Pricing'].str.extract(r'\$(\d+(?:\.\d{2})?)')

In [203]:
print("new column size ", len(df_price))

print("df size ", len(ev_station_v1))

new column size  47086
df size  47086


In [204]:
ev_station_v1['EV Pricing'] = df_price

In [205]:
ev_station_v1['EV Pricing'] = ev_station_v1['EV Pricing'].astype(float)

**4. Adding currency column**

In [206]:
ev_station_v1['currency']= "USD"

**5. Adding PK column**

In [207]:
ev_station_v1 =ev_station_v1.reset_index(drop=True)
ev_station_v1['ev_cs_dim_id'] =  ev_station_v1.index+1

**6. Adding SK Column**

In [208]:
ev_station_v1 =ev_station_v1.reset_index(drop=True)
ev_station_v1['sk_ev_dim_id'] =  ev_station_v1.index+1

**7. Adding SCD columns:**

In [209]:
ev_station_v1["row_effective_date"]= pd.to_datetime("04/21/2023")
ev_station_v1["row_expiration_date"]= pd.Series(dtype='datetime64[ns]')
ev_station_v1["row_status"]="active"

**8. Tranforming columns**

In [210]:
ev_station_v1.columns

Index(['ID', 'Status Code', 'Station Name', 'Access Code', 'City', 'State',
       'Country', 'Latitude', 'Longitude', 'Open Date', 'EV Pricing',
       'ev_cs_quantity', 'currency', 'ev_cs_dim_id', 'sk_ev_dim_id',
       'row_effective_date', 'row_expiration_date', 'row_status'],
      dtype='object')

In [211]:
ev_station_v1.rename(columns={'ID': 'charging_station_id', 
                              'Station Name' : 'station_name',
                          'Status Code': 'status_code', 
                          'Status Code': 'status_code', 
                          'EV Pricing': 'ev_price',
                          'Latitude': 'latitude',
                          'Longitude': 'longitude',
                          'Access Code' : 'access_code',
                          'Open Date' : 'activity_date',
                          'City': 'city',
                          'Country': 'country',
                          'State': 'state',
                          'Model': 'vehicle_model',
                          'quantity': 'vehicle_pop_quantity',
                          }, inplace=True)

**9. Transforming activity_date column to date type**

In [212]:
ev_station_v1["activity_date"]= ev_station_v1["activity_date"].astype('datetime64[D]')

In [213]:
ev_station_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47086 entries, 0 to 47085
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   charging_station_id  47086 non-null  int64         
 1   status_code          47086 non-null  object        
 2   station_name         47086 non-null  object        
 3   access_code          47086 non-null  object        
 4   city                 47086 non-null  object        
 5   state                47086 non-null  object        
 6   country              47086 non-null  object        
 7   latitude             47086 non-null  float64       
 8   longitude            47086 non-null  float64       
 9   activity_date        45791 non-null  datetime64[ns]
 10  ev_price             13376 non-null  float64       
 11  ev_cs_quantity       47086 non-null  int64         
 12  currency             47086 non-null  object        
 13  ev_cs_dim_id         47086 non-

------------------- 

# 3. combining two data frames to one df 

we need to merge two data frame 
we need to check if the sizes are staied the same

In [214]:
print(ev_station_v1.shape)
print(df_ev_pop.shape)

(47086, 18)
(121976, 15)


In [215]:
print('result will have ', ((18-6)+(15-6)+6+1), 'columns')#+1 for the _merge column

result will have  28 columns


In [216]:
df_ev_pop.columns

Index(['vehicle_vin', 'dol_vehicle_id', 'vehicle_make', 'vehicle_model',
       'city', 'state', 'activity_date', 'latitude', 'longitude', 'country',
       'current_reg_status', 'vehicle_pop_quantity', 'ev_car_pop_id',
       'prev_reg_status', 'reg_sta_updated_at'],
      dtype='object')

In [217]:
ev_station_v1.columns

Index(['charging_station_id', 'status_code', 'station_name', 'access_code',
       'city', 'state', 'country', 'latitude', 'longitude', 'activity_date',
       'ev_price', 'ev_cs_quantity', 'currency', 'ev_cs_dim_id',
       'sk_ev_dim_id', 'row_effective_date', 'row_expiration_date',
       'row_status'],
      dtype='object')

In [218]:
result = pd.merge(ev_station_v1,
                  df_ev_pop,
                  on = ['latitude', 'longitude','city', 'state','country', 'activity_date'],
                  how='outer',
                  indicator=True
)

In [219]:
result.shape

(169062, 28)

# 4.Creating the Location data frame 

1. extract the location data 

In [220]:
location_df= result.groupby(['latitude', 'longitude','city', 'state', 'country'], as_index=False).size()
location_df.head()

,latitude,longitude,city,state,country,size
0,-159.71288,21.96605,Kekaha,HI,US,1
1,-158.00833,21.31431,Ewa Beach,HI,US,1
2,-157.94001,21.34183,Honolulu,HI,US,1
3,-157.93084,21.38013,Aiea,HI,US,1
4,-147.72213,64.84527,Fairbanks,AK,US,1


In [221]:
location_df_distinct_city = result.groupby(['city', 'state', 'country'], as_index=False).size()

2. Add region data 

In [222]:
def get_region(state):
    regions = {
        'AL': 'South',
        'AK': 'West',
        'AZ': 'West',
        'AR': 'South',
        'CA': 'West',
        'CO': 'West',
        'CT': 'Northeast',
        'DE': 'South',
        'FL': 'South',
        'GA': 'South',
        'HI': 'West',
        'ID': 'West',
        'IL': 'Midwest',
        'IN': 'Midwest',
        'IA': 'Midwest',
        'KS': 'Midwest',
        'KY': 'South',
        'LA': 'South',
        'ME': 'Northeast',
        'MD': 'South',
        'MA': 'Northeast',
        'MI': 'Midwest',
        'MN': 'Midwest',
        'MS': 'South',
        'MO': 'Midwest',
        'MT': 'West',
        'NE': 'Midwest',
        'NV': 'West',
        'NH': 'Northeast',
        'NJ': 'Northeast',
        'NM': 'West',
        'NY': 'Northeast',
        'NC': 'South',
        'ND': 'Midwest',
        'OH': 'Midwest',
        'OK': 'South',
        'OR': 'West',
        'PA': 'Northeast',
        'RI': 'Northeast',
        'SC': 'South',
        'SD': 'Midwest',
        'TN': 'South',
        'TX': 'South',
        'UT': 'West',
        'VT': 'Northeast',
        'VA': 'South',
        'WA': 'West',
        'WV': 'South',
        'WI': 'Midwest',
        'WY': 'West',
    }
    return regions.get(state, None)


In [223]:
location_df['region']= location_df['state'].apply(get_region)

In [224]:
location_df.head()

,latitude,longitude,city,state,country,size,region
0,-159.71288,21.96605,Kekaha,HI,US,1,West
1,-158.00833,21.31431,Ewa Beach,HI,US,1,West
2,-157.94001,21.34183,Honolulu,HI,US,1,West
3,-157.93084,21.38013,Aiea,HI,US,1,West
4,-147.72213,64.84527,Fairbanks,AK,US,1,West


In [225]:
location_df = location_df.drop(columns=['size'])

In [226]:
location_df.head()

,latitude,longitude,city,state,country,region
0,-159.71288,21.96605,Kekaha,HI,US,West
1,-158.00833,21.31431,Ewa Beach,HI,US,West
2,-157.94001,21.34183,Honolulu,HI,US,West
3,-157.93084,21.38013,Aiea,HI,US,West
4,-147.72213,64.84527,Fairbanks,AK,US,West


**adding city lat and long** 

creating dstinct city,state, country df 

In [227]:
distinct_city = location_df.groupby(['city', 'state', 'country'], as_index=False).size()

In [228]:
import pandas as pd 

In [229]:
city_lat_long = pd.read_pickle('df_cities2.pkl')
city_lat_long.head()

,City,State,Country,size,city_lat,city_long
0,AIKEN,SC,US,1,33.572319,-81.618209
1,ANN ARBOR,MI,US,1,42.268157,-83.731229
2,ANTHEM,AZ,US,1,33.858395,-112.11357
3,AUBURN,WA,US,1,47.307537,-122.230181
4,Abbotsford,WI,US,1,44.946356,-90.315969


In [230]:
city_lat_long_missing = pd.read_pickle('df_cities_missing2.pkl')
print("size",len(city_lat_long_missing))
city_lat_long_missing.head()

size 2018


,City,State,Country,size,city_lat,city_long
4673,Phoenix City,AL,US,1,NaN,NaN
4674,Phoenixville,PA,US,6,NaN,NaN
4675,Picayune,MS,US,1,NaN,NaN
4676,Pickens,SC,US,1,NaN,NaN
4677,Pickerington,OH,US,2,NaN,NaN


In [231]:
city_lat_long_v3 =  pd.concat([city_lat_long, city_lat_long_missing], ignore_index=True)
print('size',len( city_lat_long_v3))
print('original distinct df', len(distinct_city))
print('difference', (6691- 6687))

size 6691
original distinct df 6687
difference 4


In [232]:
city_lat_long_v3.rename(columns={
                          'City': 'city',
                          'Country': 'country',
                          'State': 'state',
                          }, inplace=True)

In [233]:
city_lat_long_v3.columns

Index(['city', 'state', 'country', 'size', 'city_lat', 'city_long'], dtype='object')

In [234]:
distinct_city.merge(city_lat_long_v3, on=['city', 'state', 'country'], how='outer', indicator=True).query("_merge != 'both'")

,city,state,country,size_x,size_y,city_lat,city_long,_merge
6687,Bucoda,WA,US,NaN,1,46.797721,-122.868455,right_only
6688,Kapowsin,WA,US,NaN,1,46.985658,-122.22567,right_only
6689,Southworth,WA,US,NaN,2,NaN,NaN,right_only
6690,Wilkeson,WA,US,NaN,4,NaN,NaN,right_only


In [235]:
merge = pd.merge(location_df, city_lat_long_v3 , on =['city', 'state', 'country']) 

In [236]:
merge.head()

,latitude,longitude,city,state,country,region,size,city_lat,city_long
0,-159.712880,21.966050,Kekaha,HI,US,West,1,21.967281,-159.713603
1,-158.008330,21.314310,Ewa Beach,HI,US,West,3,21.315603,-158.008804
2,21.328272,-158.027542,Ewa Beach,HI,US,West,3,21.315603,-158.008804
3,21.334180,-158.023420,Ewa Beach,HI,US,West,3,21.315603,-158.008804
4,-157.940010,21.341830,Honolulu,HI,US,West,139,21.304547,-157.855676


In [237]:
print('missing cities', len(city_lat_long_missing))

missing cities 2018


In [238]:
# city_lat_long_missing_filled = city_lat_long_missing.loc[city_lat_long_missing['city_lat'].notna(), :]
# city_lat_long_missing2 = city_lat_long_missing.loc[~ city_lat_long_missing['city_lat'].notna(), :]
# city_lat_long_missing2_filled = city_lat_long_missing2.loc[city_lat_long_missing2['city_lat'].notna(), :]
# city_lat_long_missing3 = city_lat_long_missing2.loc[~ city_lat_long_missing2['city_lat'].notna(), :]
# city_lat_long_v1 = pd.concat([city_lat_long, city_lat_long_missing_filled], ignore_index=True)
# city_lat_long_v2 =  pd.concat([city_lat_long_v1, city_lat_long_missing2_filled], ignore_index=True)
# city_lat_long_v3 =  pd.concat([city_lat_long_v2, city_lat_long_missing3_filled], ignore_index=True)
# # save the DataFrame to a pickle file
# city_lat_long_missing4.to_pickle('df_cities_missing2.pkl')
# city_lat_long_v3.to_pickle('df_cities2.pkl')

In [239]:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="my_app")

# # location = geolocator.geocode(result["City"][0] + ", " + result["State"][0])

# # print(location.latitude, location.longitude)


# for index, row in city_lat_long_missing.iterrows():
#     geolocation = geolocator.geocode(row["City"] + ", " + row["State"])
#     if geolocation is None:
#         city_lat_long_missing.at[index, 'city_lat'] = "null"
#         city_lat_long_missing.at[index, 'city_long'] = 'null'
#     else: 
#         city_lat_long_missing.at[index, 'city_lat'] = geolocation.latitude
#         city_lat_long_missing.at[index, 'city_long'] = geolocation.longitude

In [240]:
location_df = merge
location_df.head()

,latitude,longitude,city,state,country,region,size,city_lat,city_long
0,-159.712880,21.966050,Kekaha,HI,US,West,1,21.967281,-159.713603
1,-158.008330,21.314310,Ewa Beach,HI,US,West,3,21.315603,-158.008804
2,21.328272,-158.027542,Ewa Beach,HI,US,West,3,21.315603,-158.008804
3,21.334180,-158.023420,Ewa Beach,HI,US,West,3,21.315603,-158.008804
4,-157.940010,21.341830,Honolulu,HI,US,West,139,21.304547,-157.855676


**3. create pk column**

In [241]:
location_df = location_df.reset_index(drop=True)
location_df['location_id']= location_df.index+1

In [242]:
result.shape

(169062, 28)

**adding scd columns**

In [243]:
import numpy as np

In [244]:
location_df['latitude_prev']= float(np.NaN)
location_df['longitude_prev']= float(np.NaN)

**merging to the fact table**

In [245]:
result = pd.merge(result, location_df, on=['latitude', 'longitude', 'city', 'state', 'country'])
result.head()

,charging_station_id,status_code,station_name,access_code,city,state,country,latitude,longitude,activity_date,...,prev_reg_status,reg_sta_updated_at,_merge,region,size,city_lat,city_long,location_id,latitude_prev,longitude_prev
0,1517.0,open,LADWP - Truesdale Center,private,Sun Valley,CA,US,34.248319,-118.387971,1999-10-15,...,NaN,NaT,left_only,West,6,NaN,NaN,20905,NaN,NaN
1,1519.0,open,LADWP - West LA District Office,private,Los Angeles,CA,US,34.052542,-118.448504,2020-02-28,...,NaN,NaT,left_only,West,1345,34.053691,-118.242766,5162,NaN,NaN
2,1520.0,temprorally_closed,Southern California Edison - Rosemead Office B...,private,Rosemead,CA,US,34.050745,-118.081014,2011-08-01,...,NaN,NaT,left_only,West,15,NaN,NaN,19785,NaN,NaN
3,75450.0,temprorally_closed,Southern California Edison - Rosemead Office B...,private,Rosemead,CA,US,34.050745,-118.081014,2016-03-15,...,NaN,NaT,left_only,West,15,NaN,NaN,19785,NaN,NaN
4,1523.0,open,Los Angeles Convention Center,public,Los Angeles,CA,US,34.040539,-118.271387,1995-08-30,...,NaN,NaT,left_only,West,1345,34.053691,-118.242766,4936,NaN,NaN


In [246]:
result.shape

(169032, 35)

In [247]:
# query example 
# result['dv_pop_quantity']=result['dv_pop_quantity'].fillna(0)
# result['quantity']=result['quantity'].fillna(0)
# result['quantity'].unique()

# summary = result.groupby(['City', 'State', 'Country']).agg({'dv_pop_quantity': 'sum','quantity': 'sum' })
# summary

---------------------------

In [248]:
result.columns

Index(['charging_station_id', 'status_code', 'station_name', 'access_code',
       'city', 'state', 'country', 'latitude', 'longitude', 'activity_date',
       'ev_price', 'ev_cs_quantity', 'currency', 'ev_cs_dim_id',
       'sk_ev_dim_id', 'row_effective_date', 'row_expiration_date',
       'row_status', 'vehicle_vin', 'dol_vehicle_id', 'vehicle_make',
       'vehicle_model', 'current_reg_status', 'vehicle_pop_quantity',
       'ev_car_pop_id', 'prev_reg_status', 'reg_sta_updated_at', '_merge',
       'region', 'size', 'city_lat', 'city_long', 'location_id',
       'latitude_prev', 'longitude_prev'],
      dtype='object')

# 5 Creating the date df 

a. extracting the date column as a new df 

In [249]:
date_df = result.loc[:, ['activity_date']].drop_duplicates( keep='first')

In [250]:
len(date_df)

2720

In [251]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2720 entries, 0 to 162999
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   activity_date  2719 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 42.5 KB


In [252]:
date_df['day'] = date_df['activity_date'].dt.day.fillna(0).astype(int)
date_df['month']= date_df['activity_date'].dt.month.fillna(0).astype(int)
date_df['year']= date_df['activity_date'].dt.year.fillna(0).astype(int)

In [253]:
date_df.head()

,activity_date,day,month,year
0,1999-10-15,15,10,1999
1,2020-02-28,28,2,2020
2,2011-08-01,1,8,2011
3,2016-03-15,15,3,2016
4,1995-08-30,30,8,1995


**b. Adding pk column**

In [254]:
date_df =date_df.reset_index(drop=True)
date_df['date_id']=  date_df.index+1

**c. merging to result fact dataframe** 

In [255]:
result = pd.merge(result, date_df, on=['activity_date'])
result.head()

,charging_station_id,status_code,station_name,access_code,city,state,country,latitude,longitude,activity_date,...,size,city_lat,city_long,location_id,latitude_prev,longitude_prev,day,month,year,date_id
0,1517.0,open,LADWP - Truesdale Center,private,Sun Valley,CA,US,34.248319,-118.387971,1999-10-15,...,6,NaN,NaN,20905,NaN,NaN,15,10,1999,1
1,1525.0,open,LADWP - John Ferraro Building,private,Los Angeles,CA,US,34.059133,-118.248589,1999-10-15,...,1345,34.053691,-118.242766,5286,NaN,NaN,15,10,1999,1
2,1552.0,open,LADWP - Harbor Generating Station,private,Wilmington,CA,US,33.770508,-118.265628,1999-10-15,...,7,NaN,NaN,17783,NaN,NaN,15,10,1999,1
3,1572.0,open,LADWP - EV Service Center,private,Los Angeles,CA,US,34.066801,-118.227605,1999-10-15,...,1345,34.053691,-118.242766,5427,NaN,NaN,15,10,1999,1
4,1589.0,open,LADWP - Palmetto Center,private,Los Angeles,CA,US,34.039736,-118.236065,1999-10-15,...,1345,34.053691,-118.242766,4922,NaN,NaN,15,10,1999,1


-----------------------  --------------------------------

# 6 ev charging facts df

In [256]:
result.columns

Index(['charging_station_id', 'status_code', 'station_name', 'access_code',
       'city', 'state', 'country', 'latitude', 'longitude', 'activity_date',
       'ev_price', 'ev_cs_quantity', 'currency', 'ev_cs_dim_id',
       'sk_ev_dim_id', 'row_effective_date', 'row_expiration_date',
       'row_status', 'vehicle_vin', 'dol_vehicle_id', 'vehicle_make',
       'vehicle_model', 'current_reg_status', 'vehicle_pop_quantity',
       'ev_car_pop_id', 'prev_reg_status', 'reg_sta_updated_at', '_merge',
       'region', 'size', 'city_lat', 'city_long', 'location_id',
       'latitude_prev', 'longitude_prev', 'day', 'month', 'year', 'date_id'],
      dtype='object')

adding pk column

In [257]:
result =result.reset_index(drop=True)
result['ev_char_fact_id']=  result.index+1

In [258]:
ev_charginging_facts = result.loc[:, ['ev_char_fact_id','ev_car_pop_id','ev_cs_dim_id','location_id','date_id','vehicle_pop_quantity', 'ev_cs_quantity','ev_price']]

In [259]:
ev_charginging_facts.head()

,ev_char_fact_id,ev_car_pop_id,ev_cs_dim_id,location_id,date_id,vehicle_pop_quantity,ev_cs_quantity,ev_price
0,1,NaN,1.0,20905,1,NaN,1.0,NaN
1,2,NaN,5.0,5286,1,NaN,1.0,NaN
2,3,NaN,8.0,17783,1,NaN,1.0,NaN
3,4,NaN,10.0,5427,1,NaN,1.0,NaN
4,5,NaN,13.0,4922,1,NaN,1.0,NaN


replace ev_car_pop_id	NaN with -1


we have to create a an fk with null values in both table when we load 

In [260]:
ev_charginging_facts['ev_car_pop_id'] = ev_charginging_facts['ev_car_pop_id'].fillna(-1)
ev_charginging_facts['location_id'] = ev_charginging_facts['location_id'].fillna(-1)

# 7 Creating the cumulative dataframe for the cumulative fact table 

in here we will group by the city lat and long and sum 

In [261]:
result.columns

Index(['charging_station_id', 'status_code', 'station_name', 'access_code',
       'city', 'state', 'country', 'latitude', 'longitude', 'activity_date',
       'ev_price', 'ev_cs_quantity', 'currency', 'ev_cs_dim_id',
       'sk_ev_dim_id', 'row_effective_date', 'row_expiration_date',
       'row_status', 'vehicle_vin', 'dol_vehicle_id', 'vehicle_make',
       'vehicle_model', 'current_reg_status', 'vehicle_pop_quantity',
       'ev_car_pop_id', 'prev_reg_status', 'reg_sta_updated_at', '_merge',
       'region', 'size', 'city_lat', 'city_long', 'location_id',
       'latitude_prev', 'longitude_prev', 'day', 'month', 'year', 'date_id',
       'ev_char_fact_id'],
      dtype='object')

In [268]:
#query example 
# result['dv_pop_quantity']=result['dv_pop_quantity'].fillna(0)
# result['quantity']=result['quantity'].fillna(0)
# result['quantity'].unique()

summary = result.groupby(['year','month','city_lat', 'city_long','city', 'state', 'country']).agg({'vehicle_pop_quantity': 'sum','ev_cs_quantity': 'sum' })
summary = summary.reset_index()

In [269]:
summary.to_pickle('cumulative_df.pkl')